# Paris Outer Rim
HyperSpy 1.3 and need to merge the Pull request #1835 and #1462

This work is part of a published article please refer to this link and cite the authors if you are using it

## Table of contents:

* [1. Blind background modelling and elemental map exctraction](#first-bullet)
* [2. Mineral map and MLLS procedure](#second-bullet)
* [3. Background and peak fitting](#third-bullet)
* [4. Pixel to pixel composition](#fourth-bullet)
* [5. Calculation of the bulk composition](#fifth-bullet)

In [2]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff

In [9]:
import imp
hs=imp.load_source("hyperspy", "C:\\Users\Pierre-Marie\Documents\GitHub\ZanettaPM\hyperspy\hyperspy/__init__.py")
hs=imp.load_source("hyperspy.api", "C:\\Users\Pierre-Marie\Documents\GitHub\ZanettaPM\hyperspy/hyperspy/api.py")

In [5]:
import hyperspy.api as hs

In [4]:
hs.hyperspy.Release.version

'1.5.2'

## Loading Data

In [10]:
s=hs.load('Signal map.hspy')

In [11]:
s.plot(True)

In [4]:
#s=hs.load('signal_size_carto_gauche.hspy')

In [15]:
s.plot(True)

In [14]:
s=s.isig[0.17:10.]

In [38]:
s.metadata.Sample.elements=[]
s.set_elements(["C", "O",'Na','K',"Mg", "Al", "Si", "Fe","Fe", "Cr", "Ni",'S','Ca','Cu','Ga','Pt','P','Cl','Mn','N','Ti','F'])

In [39]:
s.metadata.Sample.add_node('xray_lines')
s.set_lines([])
s.add_lines(['Fe_La'])

## 1. Blind background modelling and elemental map exctraction <a class="anchor" id="first-bullet"></a>

In [57]:
m=s.create_model(auto_background=False)

Here we normally use our personnal detector efficiency data. This data are sensitive and a public curve has been used

In [58]:
m.add_physical_background(detector='SuperX',quantification='Mean')

In [59]:
m.components.Bremsstrahlung.initialize()

{'Quant map and absorption correction parameters have been created'}

In [60]:
m.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |               Al_Ka |               Al_Ka |            Gaussian
   1 |               Al_Kb |               Al_Kb |            Gaussian
   2 |                C_Ka |                C_Ka |            Gaussian
   3 |               Ca_Ka |               Ca_Ka |            Gaussian
   4 |               Ca_Kb |               Ca_Kb |            Gaussian
   5 |               Ca_La |               Ca_La |            Gaussian
   6 |               Ca_Ln |               Ca_Ln |            Gaussian
   7 |               Ca_Ll |               Ca_Ll |            Gaussian
   8 |               Cl_Ka |               Cl_Ka |            Gaussian
   9 |               Cl_Kb |               Cl_Kb |            Gaussian
  10 |               Cr_Ka |               Cr_Ka |            Gaussian
  11 |               Cr_Kb |               Cr_Kb |            Gaussian
  12 |

In [61]:
To_delete = []
for j in range (6,8):
    To_delete.append(m[j].name)
for j in range (13, 16):
    To_delete.append(m[j].name)
for j in range (19, 23):
    To_delete.append(m[j].name)
for j in range (32, 36):
    To_delete.append(m[j].name)
for j in range (43, 46):
    To_delete.append(m[j].name)
for j in range (60, 69):
    To_delete.append(m[j].name)
for j in range (76, 79):
    To_delete.append(m[j].name)
m.remove(To_delete)

In [62]:
m.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |               Al_Ka |               Al_Ka |            Gaussian
   1 |               Al_Kb |               Al_Kb |            Gaussian
   2 |                C_Ka |                C_Ka |            Gaussian
   3 |               Ca_Ka |               Ca_Ka |            Gaussian
   4 |               Ca_Kb |               Ca_Kb |            Gaussian
   5 |               Ca_La |               Ca_La |            Gaussian
   6 |               Cl_Ka |               Cl_Ka |            Gaussian
   7 |               Cl_Kb |               Cl_Kb |            Gaussian
   8 |               Cr_Ka |               Cr_Ka |            Gaussian
   9 |               Cr_Kb |               Cr_Kb |            Gaussian
  10 |               Cr_La |               Cr_La |            Gaussian
  11 |               Cu_Ka |               Cu_Ka |            Gaussian
  12 |

Finally, we fit the background to the data. The background cannot be negative, in consequence we fix bounds. 

In [63]:
m.fit_background(bounded=True, windows_sigma=([3,3]))

In [64]:
m.components.Bremsstrahlung.coefficients.assign_current_value_to_all()

In [53]:
m.fit_background(kind='multi',bounded=True, windows_sigma=([3,3]))

In [56]:
m.plot(True)

And we fit the gaussians using the linear fit proposed by thomas aarholt (already merged in "v1.3_background_and_linear")
https://github.com/hyperspy/hyperspy/pull/1462

In [54]:
m.fix_background()

In [66]:

m.set_parameters_value('A',1)
m.multifit(fitter='linear',bounded=True,grad=True)

KeyboardInterrupt: 

In [67]:
m.plot(plot_components=True)

We extract the different elementary map in order to use clustering methods to classify pixel of composition fields
Here we plot the Mg map corresponding to the integrated peak after the background removing

In [68]:
elemap=m.get_lines_intensity()
plt.figure()
plt.imshow(elemap[5],cmap='jet')
plt.title('Mg map, background corrected, in number of counts')
plt.colorbar()

In [37]:
a=s.metadata.Sample.xray_lines

In [42]:
for i in range(0,len(elemap)):
    b=elemap[i]
    b=np.float32(b)
    tiff.imsave('C:\Users\Pierre-Marie\Documents\GitHub\TEM-Phase-map-exemple---Paris-meteorite\elemap\ ' + a[i] + '.tif',(b))
    np.savetxt('C:\Users\Pierre-Marie\Documents\GitHub\TEM-Phase-map-exemple---Paris-meteorite\elemap\ ' + a[i] + '.txt',(b))

In [14]:
m.load_parameters_from_file('model.npz')

In [20]:
m.save_parameters2file('model')

## 2. Mineral map and MLLS procedure <a class="anchor" id="second-bullet"></a>

A simple phase map is first obtained using XmapTools software based on the elemental map extracted above. This procedure is used to find the differente phases. It is not obligatory but help to get a final map faster.

Clustering methods exist in python and have been proposed into hyperspy: 
https://github.com/hyperspy/hyperspy/pull/1353

Here to continue the example, a file containing the value of the map is provided in the repository of this tutorial

In [70]:
carto=np.loadtxt('Map.txt')

In [71]:
phases=['Al-rich_amorphous','Holes','Carbonates', 'Chromite', 'Platinium', 'FeS', 'Fib_boundary2','Pyroxene ','Fayalite','Fe_amorphous','Pentlandite','Mg_amorphous','Olivine']

We first plot the phase map

In [72]:
N=len(phases)

In [73]:
plt.figure()
plt.imshow(carto,plt.get_cmap('jet', N))
cb = plt.colorbar()

ticks = [1.5]
for i in range(len(phases)-1):
    ticks.append(0)
    ticks[i+1]=ticks[i]+(N-2)/(N-1)
ticks

cb.set_ticks([ticks])
cb.set_ticklabels(phases)


Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backends\backend_qt5.py", line 505, in _draw_idle
    self.draw()
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py", line 388, in draw
    self.figure.draw(self.renderer)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\artist.py", line 38, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\figure.py", line 1709, in draw
    renderer, self, artists, self.suppressComposite)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\image.py", line 135, in _draw_list_compositing_images
    a.draw(renderer)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\artist.py", line 38, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\axes\_base.py", line 2647, in draw
    mimage._dra

In [13]:
s.plot(True)

In [74]:
carto.shape

(132, 152)

In [75]:
s.axes_manager

<Axes manager, axes: (152, 132|492)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    152 |     61 | -0.0045 |   0.051 |     um 
               y |    132 |     39 | 0.00093 |   0.051 |     um 
---------------- | ------ | ------ | ------- | ------- | ------ 
    X-ray energy |    492 |        |    0.17 |    0.02 |    keV

In [76]:
carto.shape

(132, 152)

We create a new model that will contain the refence spectra of the MLLS procedure

In [77]:
m2 = s.create_model(False,False)

Ideally we use selected spectrum from the reference grains. Here a simple mean spectrum from all pixels of each phase is calculated

In [78]:
carto=np.array(carto.data)
a=int(np.max(carto))
Sref=np.zeros([a,m2._signal.axes_manager[-1].size])
component=dict()
for i in range (0,a):
    s2=s.data[carto==i+1]
    s2=hs.signals.EDSSEMSpectrum(s2)
    s2=s2.mean()
    s2.get_calibration_from(s) 
    s2.metadata=s.metadata 
    Sref[i]=s2
    component[i]=s2
Sref=hs.signals.EDSSEMSpectrum(Sref)
Sref.get_calibration_from(s) 
Sref.metadata=s.metadata 

One of the reference spectrum is plotted

We add the reference spectra as component of the new model

In [79]:
Sref.plot(True)

In [80]:
for i in range (0,a):
    g=hs.model.components1D.ScalableFixedPattern(component[i])
    m2.extend([g])

In [81]:
m2.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   1 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   2 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   3 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   4 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   5 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   6 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   7 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   8 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   9 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
  10 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
  11 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
  12 |

The only free parameter is the Yscale, other are fixed. The yscale is bounded between 0 and 1 to obtain a percentage of the different phases

In [82]:
for i in range (0,len(m2)):
    m2.set_parameters_not_free([i], parameter_name_list=['xscale','shift'])
    m2.set_parameters_value('yscale',value=0.1,component_list=[i])
    m2[i].yscale.bmin=0
    m2[i].yscale.bmax=1

In [83]:
m2.load_parameters_from_file('model_mlls.npz')

In [17]:
m2.multifit(bounded=True, grad=True)

KeyboardInterrupt: 

We plot maps of "proportion in each pixel" of the different phases

In [84]:
threshold=[0.35,0.6,0.4,0.3,0.6,0.3,0.1,0.6,0.2,0.3,0.5,0.1,0.7]

In [86]:
mllsmap = hs.signals.Signal2D(np.zeros((len(phases), s.data.shape[0], s.data.shape[1])))
for i in range(0,len(m2)):
    mllsmap.data[i,:,:]=m2[i].yscale.map['values']
    mask=(mllsmap.data[i,:,:])<threshold[i]
    mllsmap.data[i,:,:][mask]=0
mllsmap.plot(cmap='jet')

In [88]:
colorscale=([[0.90664343, 0.74466896, 0.86459331],
       [0, 0, 0],
       [0, 0,1],
       [0.8055239 , 0.12485083, 0.05532277],
       [1,1,1],
       [0,1,0],
       [1,1,1],
       [0.8,0.4,0.3],
       [1,0.6,0],
       [0., 0.6, 0.8],
       [0.9,1,0.1],
       [0.1,0.8,0.9],
       [0.9,0,0.8]])

In [89]:
from skimage import color

RGB = []
cmap=plt.cm.hsv
cmaplist=np.zeros([N,4])
RGB_final = 0
colorscale = np.array(colorscale)


for i in range (0,N):
    cmaplist[i]=(colorscale[i,:][0],colorscale[i,:][1],colorscale[i,:][2],1)
#create the nex map
cmap = cmap.from_list('custum cmap', cmaplist,N)


for i in range (N):
    RGB.append(0)
    RGB[i] = color.gray2rgb(mllsmap.data[i,:,:])
    for j in range (3):
        RGB[i][:,:,j] = RGB[i][:,:,j]*colorscale[i][j]
    RGB_final += RGB[i]

plt.figure()
plt.imshow(RGB_final,cmap=cmap)
cb=plt.colorbar()

ticks = [0.030]
for i in range(len(phases)-1):
    ticks.append(0)
    ticks[i+1]=ticks[i]+((N-2)/(N-1))/11.9
ticks

cb.set_ticks([ticks])
cb.set_ticklabels(phases)



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backends\backend_qt5.py", line 505, in _draw_idle
    self.draw()
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py", line 388, in draw
    self.figure.draw(self.renderer)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\artist.py", line 38, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\figure.py", line 1709, in draw
    renderer, self, artists, self.suppressComposite)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\image.py", line 135, in _draw_list_compositing_images
    a.draw(renderer)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\artist.py", line 38, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\axes\_base.py", line 2647, in draw
    mimage._dra

In [96]:
a=mllsmap.data[9,:,:]>0 
b=mllsmap.data[11,:,:]>0 
a.astype(dtype=int)
b.astype(dtype=int)
M1=a+b

M1=hs.signals.BaseSignal(M1)
M1.plot()

In [39]:
M1.save('mask_amorphous.hspy')

In [23]:
m2.save_parameters2file('model_mlls')

# 3. Background and peak fitting <a class="anchor" id="third-bullet"></a>

We have now corrected our modal abundances. We will now fit correctly the background and the peak in more detail in order to get the intensity needed for the quantification

In [100]:
#S2=s.rebin([152/2, 132/2,492])

In [98]:
#S2=hs.save('signal_quanti.hspy')

In [ ]:
S2=hs.load('signal_quanti.hspy')

In [103]:
S2

<EDSTEMSpectrum, title: EDS, dimensions: (76, 66|492)>

In [104]:
S2.metadata.Sample.elements=[]
S2.set_elements(["C", "O",'Na','K',"Mg", "Al", "Si", "Fe","Fe", "Cr", "Ni",'S','Ca','Cu','Ga','Pt','P','Cl','Mn','N','Ti','F'])

In [105]:
S2.metadata.Sample.add_node('xray_lines')
S2.set_lines([])
S2.add_lines(['Fe_La'])

In [106]:
S2.metadata.Acquisition_instrument.TEM.Detector.EDS.elevation_angle = 18.

In [107]:
S2.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 18.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       └── number_of_frames = 3004
│       ├── Stage
│       │   ├── tilt_alpha = 0.00
│       │   ├── tilt_beta = 0.00
│       │   ├── x = -0.000
│       │   ├── y = -0.000
│       │   └── z = -0.000
│       ├── beam_energy = 300.0
│       ├── camera_length = 115.9
│       ├── magnification = 5000.0
│       └── microscope = Titan
├── General
│   ├── date = 2018-05-03
│   ├── original_filename = SI EDS-HAADF 1644 20180503.emd
│   ├── time = 16:44:48
│   ├── time_zone = Paris, Madrid (heure dété)
│   └── title = EDS
├── Sample
│   ├── elements = ['Al', 'C', 'Ca', 'Cl', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'K', 'Mg', 'Mn', 'N', 'Na', 'Ni', 'O', 'P', 'Pt', 'S', 'Si', 'Ti']
│   └── xray_lines <list>
│       ╠══ [0] = Al_Ka
│       ╠══ [1] = C_Ka
│       ╠══ [10] = K_Ka
│       

In [120]:
import numpy as np
#superx=np.loadtxt('Det_eff.txt') #insert your ow
#superx = 0.6*superx

m2 = S2.create_model(auto_background = False)

In [121]:
m2.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |               Al_Ka |               Al_Ka |            Gaussian
   1 |               Al_Kb |               Al_Kb |            Gaussian
   2 |                C_Ka |                C_Ka |            Gaussian
   3 |               Ca_Ka |               Ca_Ka |            Gaussian
   4 |               Ca_Kb |               Ca_Kb |            Gaussian
   5 |               Ca_La |               Ca_La |            Gaussian
   6 |               Ca_Ln |               Ca_Ln |            Gaussian
   7 |               Ca_Ll |               Ca_Ll |            Gaussian
   8 |               Cl_Ka |               Cl_Ka |            Gaussian
   9 |               Cl_Kb |               Cl_Kb |            Gaussian
  10 |               Cr_Ka |               Cr_Ka |            Gaussian
  11 |               Cr_Kb |               Cr_Kb |            Gaussian
  12 |

In [122]:
To_delete = []
for j in range (6,8):
    To_delete.append(m2[j].name)
for j in range (13, 16):
    To_delete.append(m2[j].name)
for j in range (19, 23):
    To_delete.append(m2[j].name)
for j in range (32, 36):
    To_delete.append(m2[j].name)
for j in range (43, 46):
    To_delete.append(m2[j].name)
for j in range (60, 69):
    To_delete.append(m2[j].name)
for j in range (76, 79):
    To_delete.append(m2[j].name)
m2.remove(To_delete)

In [123]:
m2.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |               Al_Ka |               Al_Ka |            Gaussian
   1 |               Al_Kb |               Al_Kb |            Gaussian
   2 |                C_Ka |                C_Ka |            Gaussian
   3 |               Ca_Ka |               Ca_Ka |            Gaussian
   4 |               Ca_Kb |               Ca_Kb |            Gaussian
   5 |               Ca_La |               Ca_La |            Gaussian
   6 |               Cl_Ka |               Cl_Ka |            Gaussian
   7 |               Cl_Kb |               Cl_Kb |            Gaussian
   8 |               Cr_Ka |               Cr_Ka |            Gaussian
   9 |               Cr_Kb |               Cr_Kb |            Gaussian
  10 |               Cr_La |               Cr_La |            Gaussian
  11 |               Cu_Ka |               Cu_Ka |            Gaussian
  12 |

In [124]:
m2.components.Fe_La.A.twin = None
m2.components.Fe_Ll.A.twin = None

In [125]:
m2.add_physical_background(detector='SuperX', absorption_model='quadrilateral')#, quantification = SimplQuant)
m2.components.Bremsstrahlung.initialize()

{'Quant map and absorption correction parameters have been created'}

In [126]:
m2.fit_background(kind = 'multi', bounded = True, windows_sigma=(3, 3))

KeyboardInterrupt: 

In [127]:
m2.set_parameters_value('A',1)
m2.multifit(fitter='linear',bounded=True)

AttributeError: Not all components are linear. Fit with a different fitter or set non-linear `parameters.free = False`. These components are nonlinear:[<Bremsstrahlung (Physical_background component)>]

In [48]:
m2.plot(True)

In [49]:
m2.save_parameters2file('model_quanti.npz')

In [128]:
S2.set_lines([])
S2.add_lines(['Fe_La'])

In [129]:
result = m2.get_lines_intensity(xray_lines = 'from_metadata', plot_result=False)

In [130]:
result

[<BaseSignal, title: Intensity of Al_Ka at 1.49 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of C_Ka at 0.28 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of Ca_Ka at 3.69 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of Cl_Ka at 2.62 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of Cr_Ka at 5.41 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of Cu_Ka at 8.05 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of F_Ka at 0.68 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of Fe_Ka at 6.40 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of Fe_La at 0.70 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of Ga_Ka at 9.25 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of K_Ka at 3.31 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of Mg_Ka at 1.25 keV from 

In [53]:
for i in range (len(result)):
    result[i].save(r'C:\\Your Directory'+ result[i].metadata.General.title +'.hspy',overwrite=True)

## 4. Pixel to Pixel composition <a class="anchor" id="fourth-bullet"></a>

We use our in house Cliff Lorimer + absorption correction model to calculate the composition of each pixel of the amorphous silicate
These in house code can be found here: https://github.com/CorentinLG/CL_quant

In [1]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff
import hyperspy.api as hs

In [2]:
import sys
sys.path.append(r'D:\Users\Documents\GitHub\CL_quant')
from CL_Quant import*

In [3]:
s=hs.load('signal_quanti.hspy')

In [4]:
s.set_lines([])
s.add_lines(['Fe_La'])

In [5]:
result=hs.load(r'C:\\Your Directory '+ '*.hspy')

In [15]:
result

[<BaseSignal, title: Intensity of Al_Ka at 1.49 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of C_Ka at 0.28 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of Ca_Ka at 3.69 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of Cl_Ka at 2.62 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of Cr_Ka at 5.41 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of Cu_Ka at 8.05 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of F_Ka at 0.68 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of Fe_Ka at 6.40 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of Fe_La at 0.70 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of Ga_Ka at 9.25 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of K_Ka at 3.31 keV from EDS, dimensions: (76, 66|)>,
 <BaseSignal, title: Intensity of Mg_Ka at 1.25 keV from 

In [9]:
M1=hs.load('mask_amorphous.hspy')

In [10]:
M1.plot()

In [11]:
M1=M1.rebin([152/2,132/2])

In [12]:
M1 = M1==4
M1.plot()

In [13]:
val = [3, 0, 2, -1, 2.66, 0, -1, 2.66, 2.66, 0, 1, 2, 2, 0, 1, 2, -2, -3, 0, -2, 4, 3]

In [14]:

print(s.metadata.Sample.elements),


for i in range (len(s.metadata.Sample.xray_lines)):
    print(s.metadata.Sample.xray_lines[i],val[i])

['Al', 'C', 'Ca', 'Cl', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'K', 'Mg', 'Mn', 'N', 'Na', 'Ni', 'O', 'P', 'Pt', 'S', 'Si', 'Ti']
Al_Ka 3
C_Ka 0
Ca_Ka 2
Cl_Ka -1
Cr_Ka 2.66
Cu_Ka 0
F_Ka -1
Fe_Ka 2.66
Fe_La 2.66
Ga_Ka 0
K_Ka 1
Mg_Ka 2
Mn_Ka 2
N_Ka 0
Na_Ka 1
Ni_Ka 2
O_Ka -2
P_Ka -3
Pt_La 0
S_Ka -2
Si_Ka 4
Ti_Ka 3


In [16]:
%%time

"""Quantification based on M1"""



result_cor = correct_result(result)
factors = kfactors(result)
val = val = [3, 0, 2, -1, 2.66, 0, -1, 2.66, 2.66, 0, 1, 2, 2, 0, 1, 2, -2, -3, 0, -2, 4, 3]
Quant, H2O, mt, Dev = absorption_correction_auto (result_cor, s, factors, line1 = 'Fe_Ka', line2 = 'Fe_La', Elt_rat = 1, d = 2.9, t = 150, tilt_stage = 0, navigation_mask = ~M1, Crit=0.01, water=True, valence = val)

elts [['Al'], ['C'], ['Ca'], ['Cl'], ['Cr'], ['Cu'], ['F'], ['Fe'], ['Fe'], ['Ga'], ['K'], ['Mg'], ['Mn'], ['N'], ['Na'], ['Ni'], ['O'], ['P'], ['Pt'], ['S'], ['Si'], ['Ti']]
number of pixels to deal with:  1129 total number of pixels:  5016


C:\Anaconda3\lib\site-packages\hyperspy\misc\eds\utils.py:447: RuntimeWarning: divide by zero encountered in double_scalars
  / intensities[i] / kfactors[i]
C:\Anaconda3\lib\site-packages\hyperspy\misc\eds\utils.py:447: RuntimeWarning: overflow encountered in double_scalars
  / intensities[i] / kfactors[i]
C:\Anaconda3\lib\site-packages\hyperspy\misc\eds\utils.py:383: RuntimeWarning: invalid value encountered in greater
  index = np.where(intens[:, i] > min_intensity)[0]
D:\Users\Documents\GitHub\CL_quant\absorption_correction_auto.py:86: RuntimeWarning: invalid value encountered in true_divide
  rat = SubQuant[0].data/(Elt_rat*SubQuant[1].data)
D:\Users\Documents\GitHub\CL_quant\absorption_correction_auto.py:88: RuntimeWarning: invalid value encountered in true_divide
  Dev = (SubQuant[0].data - Elt_rat * SubQuant[1].data)/SubQuant[0].data


line1/line2 deviation (%) = 22.04840621668063   processing next iteration
line1/line2 deviation (%) = 12.676834699959127   processing next iteration
line1/line2 deviation (%) = 7.997761403731639   processing next iteration
line1/line2 deviation (%) = 5.291031205207215   processing next iteration
line1/line2 deviation (%) = 3.5821725790075623   processing next iteration
line1/line2 deviation (%) = 2.487310163871302   processing next iteration
line1/line2 deviation (%) = 1.7495179919785102   processing next iteration
line1/line2 deviation (%) = 1.2415926161474715   processing next iteration
line1/line2 deviation (%) = 0.8809684556779579   processing next iteration
line1/line2 deviation (%) = 0.6273313995376226   processing next iteration
line1/line2 deviation (%) = 0.4464634763471118   processing next iteration
line1/line2 deviation (%) = 0.32029851339361726   processing next iteration
line1/line2 deviation (%) = 0.2298561114626113   processing next iteration
line1/line2 deviation (%) = 

C:\Anaconda3\lib\site-packages\hyperspy\misc\material.py:42: RuntimeWarning: invalid value encountered in true_divide
  atomic_percent[i] /= sum_weight
C:\Anaconda3\lib\site-packages\hyperspy\misc\material.py:120: RuntimeWarning: invalid value encountered in true_divide
  weight_percent[i] /= sum_atomic


Water computed
Wall time: 21min 48s


In [22]:
for i in range (len(Quant)):
    Quant[i].save(r'C:\\Your Directory\Quant\ '+ Quant[i].metadata.General.title + str(i) +'.hspy',overwrite=True)
    tiff.imsave(r'C:\\Your Directory\Quant\ '+ Quant[i].metadata.General.title + str(i) +'.tiff',(Quant[i].data))
    np.savetxt(r'C:\\Your Directory\Quant\ '+ Quant[i].metadata.General.title + str(i) +'.txt',(Quant[i].data))

In [19]:
Quant

[<BaseSignal, title: atomic percent of Al, dimensions: (76, 66|)>,
 <BaseSignal, title: atomic percent of C, dimensions: (76, 66|)>,
 <BaseSignal, title: atomic percent of Ca, dimensions: (76, 66|)>,
 <BaseSignal, title: atomic percent of Cl, dimensions: (76, 66|)>,
 <BaseSignal, title: atomic percent of Cr, dimensions: (76, 66|)>,
 <BaseSignal, title: atomic percent of Cu, dimensions: (76, 66|)>,
 <BaseSignal, title: atomic percent of F, dimensions: (76, 66|)>,
 <BaseSignal, title: atomic percent of Fe, dimensions: (76, 66|)>,
 <BaseSignal, title: atomic percent of Fe, dimensions: (76, 66|)>,
 <BaseSignal, title: atomic percent of Ga, dimensions: (76, 66|)>,
 <BaseSignal, title: atomic percent of K, dimensions: (76, 66|)>,
 <BaseSignal, title: atomic percent of Mg, dimensions: (76, 66|)>,
 <BaseSignal, title: atomic percent of Mn, dimensions: (76, 66|)>,
 <BaseSignal, title: atomic percent of N, dimensions: (76, 66|)>,
 <BaseSignal, title: atomic percent of Na, dimensions: (76, 66|)>,

In [24]:
data_output(Quant, result_cor,H2O, mt_List=mt,density_or_thickness=2.9, name = 'Quanti amorphous.xlsx')

In [17]:
plt.figure()
plt.imshow(H2O, cmap = 'jet')
plt.colorbar()
print(np.nanstd(H2O[M1]))
print(np.nanmean(H2O[M1]))

3.194373704917187
7.103450266245317


## 5. Bulk composition<a class="anchor" id="fifth-bullet"></a>

### Model

In [1]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff

In [2]:
import imp
hs=imp.load_source("hyperspy", "C:\\Users\Pierre-Marie\Documents\GitHub\ZanettaPM\hyperspy\hyperspy/__init__.py")
hs=imp.load_source("hyperspy.api", "C:\\Users\Pierre-Marie\Documents\GitHub\ZanettaPM\hyperspy/hyperspy/api.py")

In [3]:
s=hs.load('signal quanti.hspy')

In [8]:
s.plot(True)

In [4]:
s.metadata.Sample.elements=[]
s.set_elements(["C", "O",'Na','K',"Mg", "Al", "Si", "Fe","Fe", "Cr", "Ni",'S','Ca','Cu','Ga','Pt','P','Cl','Mn','N','Ti','F'])

In [5]:
M1=hs.load('mask_amorphous.hspy')

In [7]:
M1=M1.rebin([152/2,132/2])

In [8]:
M1 = M1>2
M1.plot()

In [9]:
a=s.data[M1]
a=hs.signals.EDSTEMSpectrum(a)
#a.sum().plot()
a=a.sum()

In [10]:
a.metadata=s.metadata
a.get_calibration_from(s)

In [11]:
a.plot(True)

In [12]:
a.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 18.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       └── number_of_frames = 3004
│       ├── Stage
│       │   ├── tilt_alpha = 0.00
│       │   ├── tilt_beta = 0.00
│       │   ├── x = -0.000
│       │   ├── y = -0.000
│       │   └── z = -0.000
│       ├── beam_energy = 300.0
│       ├── camera_length = 115.9
│       ├── magnification = 5000.0
│       └── microscope = Titan
├── General
│   ├── date = 2018-05-03
│   ├── original_filename = SI EDS-HAADF 1644 20180503.emd
│   ├── time = 16:44:48
│   ├── time_zone = Paris, Madrid (heure dété)
│   └── title = EDS
├── Sample
│   ├── elements = ['Al', 'C', 'Ca', 'Cl', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'K', 'Mg', 'Mn', 'N', 'Na', 'Ni', 'O', 'P', 'Pt', 'S', 'Si', 'Ti']
│   └── xray_lines <list>
│       ╠══ [0] = Al_Ka
│       ╠══ [1] = C_Ka
│       ╠══ [10] = K_Ka
│       

In [13]:
import numpy as np
#superx=np.loadtxt('Det_eff.txt')
#superx = 0.6*superx

m2 = a.create_model(auto_background = False)

In [14]:
m2.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |               Al_Ka |               Al_Ka |            Gaussian
   1 |               Al_Kb |               Al_Kb |            Gaussian
   2 |                C_Ka |                C_Ka |            Gaussian
   3 |               Ca_Ka |               Ca_Ka |            Gaussian
   4 |               Ca_Kb |               Ca_Kb |            Gaussian
   5 |               Ca_La |               Ca_La |            Gaussian
   6 |               Ca_Ln |               Ca_Ln |            Gaussian
   7 |               Ca_Ll |               Ca_Ll |            Gaussian
   8 |               Cl_Ka |               Cl_Ka |            Gaussian
   9 |               Cl_Kb |               Cl_Kb |            Gaussian
  10 |               Cr_Ka |               Cr_Ka |            Gaussian
  11 |               Cr_Kb |               Cr_Kb |            Gaussian
  12 |

In [15]:
To_delete = []
for j in range (6,8):
    To_delete.append(m2[j].name)
for j in range (13, 16):
    To_delete.append(m2[j].name)
for j in range (19, 23):
    To_delete.append(m2[j].name)
for j in range (32, 36):
    To_delete.append(m2[j].name)
for j in range (43, 46):
    To_delete.append(m2[j].name)
for j in range (60, 69):
    To_delete.append(m2[j].name)
for j in range (76, 79):
    To_delete.append(m2[j].name)
m2.remove(To_delete)

In [16]:
m2.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |               Al_Ka |               Al_Ka |            Gaussian
   1 |               Al_Kb |               Al_Kb |            Gaussian
   2 |                C_Ka |                C_Ka |            Gaussian
   3 |               Ca_Ka |               Ca_Ka |            Gaussian
   4 |               Ca_Kb |               Ca_Kb |            Gaussian
   5 |               Ca_La |               Ca_La |            Gaussian
   6 |               Cl_Ka |               Cl_Ka |            Gaussian
   7 |               Cl_Kb |               Cl_Kb |            Gaussian
   8 |               Cr_Ka |               Cr_Ka |            Gaussian
   9 |               Cr_Kb |               Cr_Kb |            Gaussian
  10 |               Cr_La |               Cr_La |            Gaussian
  11 |               Cu_Ka |               Cu_Ka |            Gaussian
  12 |

In [17]:
m2.components.Fe_La.A.twin = None
m2.components.Fe_Ll.A.twin = None

In [18]:
m2.add_physical_background(detector='SuperX', absorption_model='quadrilateral',TOA=1)#, quantification = SimplQuant)
m2.components.Bremsstrahlung.initialize()

{'Quant map and absorption correction parameters have been created'}

In [20]:
m2.fit_background(bounded = True, windows_sigma=(3, 3))

In [21]:
m2.plot(True)

In [22]:
# m2.set_parameters_value('A',1)
m2.fit(bounded=True)

In [23]:
s.set_lines([])
s.add_lines(['Fe_La'])

In [24]:
result = m2.get_lines_intensity(xray_lines = 'from_metadata', plot_result=False)

In [25]:
result

[<BaseSignal, title: Intensity of Al_Ka at 1.49 keV from EDS, dimensions: (|1)>,
 <BaseSignal, title: Intensity of C_Ka at 0.28 keV from EDS, dimensions: (|1)>,
 <BaseSignal, title: Intensity of Ca_Ka at 3.69 keV from EDS, dimensions: (|1)>,
 <BaseSignal, title: Intensity of Cl_Ka at 2.62 keV from EDS, dimensions: (|1)>,
 <BaseSignal, title: Intensity of Cr_Ka at 5.41 keV from EDS, dimensions: (|1)>,
 <BaseSignal, title: Intensity of Cu_Ka at 8.05 keV from EDS, dimensions: (|1)>,
 <BaseSignal, title: Intensity of F_Ka at 0.68 keV from EDS, dimensions: (|1)>,
 <BaseSignal, title: Intensity of Fe_Ka at 6.40 keV from EDS, dimensions: (|1)>,
 <BaseSignal, title: Intensity of Fe_La at 0.70 keV from EDS, dimensions: (|1)>,
 <BaseSignal, title: Intensity of Ga_Ka at 9.25 keV from EDS, dimensions: (|1)>,
 <BaseSignal, title: Intensity of K_Ka at 3.31 keV from EDS, dimensions: (|1)>,
 <BaseSignal, title: Intensity of Mg_Ka at 1.25 keV from EDS, dimensions: (|1)>,
 <BaseSignal, title: Intensity 

In [26]:
for i in range (len(result)):
    result[i].save(r'C:\\Your Directory'+ result[i].metadata.General.title +'.hspy',overwrite=True)

In [ ]:
s.save('amorphous mean')

### Quantification

In [1]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff
import hyperspy.api as hs

In [2]:
import sys
sys.path.append(r'D:\Users\Documents\GitHub\CL_quant')
from CL_Quant import*

ModuleNotFoundError: No module named 'CL_Quant'

In [ ]:
s=hs.load('amorphous mean.hspy')

In [ ]:
s.add_lines()

In [ ]:
result=hs.load(r'Your Directory\ '+ '*.hspy')

In [ ]:
result

In [ ]:
val = [3, 0, 2, -1, 2, 0, -1, 2, 0, 1, 2, 2, 0, 1, 2, -2, -3, 0, -2, 4, 3]
print(s.metadata.Sample.elements),


for i in range (len(s.metadata.Sample.xray_lines)):
    print(s.metadata.Sample.xray_lines[i],val[i])

In [7]:
%%time

"""Quantification based on M1"""



result_cor = correct_result(result)
factors = kfactors(result)
val = val = [3, 0, 2, -1, 2.66, 0, -1, 2.66, 2.66, 0, 1, 2, 2, 0, 1, 2, -2, -3, 0, -2, 4, 3]
Quant, H2O, mt, Dev = absorption_correction_auto (result_cor, s, factors, line1 = 'Fe_Ka', line2 = 'Fe_La', Elt_rat = 1, d = 2.9, t = 150, tilt_stage = 0, Crit=0.01, water=True, valence = val)

elts [['Al'], ['C'], ['Ca'], ['Cl'], ['Cr'], ['Cu'], ['F'], ['Fe'], ['Fe'], ['Ga'], ['K'], ['Mg'], ['Mn'], ['N'], ['Na'], ['Ni'], ['O'], ['P'], ['Pt'], ['S'], ['Si'], ['Ti']]
number of pixels to deal with:  1 total number of pixels:  1


C:\Anaconda3\lib\site-packages\hyperspy\misc\eds\utils.py:447: RuntimeWarning: divide by zero encountered in double_scalars
  / intensities[i] / kfactors[i]


line1/line2 deviation (%) = 16.723236857083812   processing next iteration
line1/line2 deviation (%) = 2.7875429883164284   processing next iteration
line1/line2 deviation (%) = 0.7173375911509156   processing next iteration
line1/line2 deviation (%) = 0.01003242111208743   processing next iteration
line1/line2 deviation (%) = 0.03520460046545045   processing next iteration
line1/line2 deviation (%) = 0.05565927557579559   processing next iteration
line1/line2 deviation (%) = 0.04096855460331422   processing next iteration
line1/line2 deviation (%) = 0.02655240783369538   processing next iteration
line1/line2 deviation (%) = 0.016501016313204993   processing next iteration
line1/line2 deviation (%) = 0.010263687012681518   processing next iteration
line1/line2 deviation (%) = 0.006448170587027541   processing next iteration
Water computed
Wall time: 3min 18s


In [8]:
H2O

array([7.79612917])

In [8]:
data_output(Quant, result_cor,H2O, mt_List=mt,density_or_thickness=2.9, name = 'amorphous_quanti_mean.xlsx')

Thanks for using this code 